- get tokenizer
- get model
- make LoRa model
- get dataset
- prepare dataset and dataloaders
- basic training loop with accelerate
- do stuff similar to NER_fewnerd_LoRA




# CausalLM with `open-orca` and LoRA

In [2]:
import transformers
from datasets import load_dataset, DatasetDict

In [3]:
ds = load_dataset("Open-Orca/OpenOrca")
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# small, medium and large
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small") 
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

/home/matthias/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm here


>> User: OK, whatever..


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm here


In [6]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print(f"DialoGPT: {format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))}")

>> User: How have you been?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm here


>> User: 


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


...

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm here


>> User: Yes, you're doing a really great job. \s


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm here


In [4]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

NameError: name 'tokenizer' is not defined

In [ ]:
# trying it out 2
def tokenize2(element):
    outputs = tokenizer(
        element["question"],
        padding=True,
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
        return_length=True,
    )
    return outputs
try2_dataset = (
    load_dataset("Open-Orca/OpenOrca")["train"]
    .select(range(100000)) # use subset of data
    .filter(lambda item: isinstance(item["response"], str)) # ensure the response is a string
    .filter(lambda item: item["response"][0].isdigit() or item["response"][0].isalpha()) # .isdigit => digit, .isalpha => letter
    .map(tokenize2) # tokenize
    .map(lambda item: {"input_ids": item["input_ids"][0]}) # remove nesting
    .remove_columns(["id", "system_prompt", "length", "overflow_to_sample_mapping", "attention_mask", "question", "response"])
)
try2_dataset